In [1]:
# train_utils.py

import torch
from torch import nn
from torch.autograd import Variable
import copy
import math

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Embeddings(nn.Module):
    '''
    Usual Embedding layer with weights multiplied by sqrt(d_model)
    '''
    def __init__(self,vocab_size , d_model):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        #self.lut = nn.Embedding.from_pretrained(embeddings, freeze=True)
        
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))
        pe[:, 1::2] = torch.cos(torch.as_tensor(position.numpy() * div_term.unsqueeze(0).numpy()))#torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)],
                         requires_grad=False)
        return self.dropout(x)
    

In [2]:
# config.py

class Config(object):
    N = 6 #6 in Transformer Paper
    d_model = 512 #512 in Transformer Paper
    d_ff = 2048 #2048 in Transformer Paper
    h = 8
    dropout = 0.1
    output_size = 4
    lr = 0.0003
    max_epochs = 200
    batch_size = 1
    max_sen_len = 50

In [31]:
# attention.py

import torch
from torch import nn
import math
import torch.nn.functional as F


def attention(query, key, value, mask=None, dropout=None):
    "Implementation of Scaled dot product attention"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn

class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), self.h)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Multi-head attention"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [3]:
class Encoder(nn.Module):
    '''
    Transformer Encoder

    It is a stack of N layers.
    '''
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class EncoderLayer(nn.Module):
    '''
    An encoder layer

    Made up of self-attention and a feed forward layer.
    Each of these sublayers have residual and layer norm, implemented by SublayerOutput.
    '''
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer_output = clones(SublayerOutput(size, dropout), 2)
        self.size = size

    def forward(self, x, mask=None):
        "Transformer Encoder"
        x = self.sublayer_output[0](x, lambda x: self.self_attn(x, x, x, mask)) # Encoder self-attention
        return self.sublayer_output[1](x, self.feed_forward)

In [4]:
# sublayer.py

import torch
from torch import nn

class LayerNorm(nn.Module):
    "Construct a layer normalization module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerOutput(nn.Module):
    '''
    A residual connection followed by a layer norm.
    '''
    def __init__(self, size, dropout):
        super(SublayerOutput, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [5]:
# feed_forward.py

from torch import nn
import torch.nn.functional as F

class PositionwiseFeedForward(nn.Module):
    "Positionwise feed-forward network."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        "Implements FFN equation."
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [6]:
import pandas as pd
import json
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
def load_data_df(train_file):
    with open(train_file,'r') as f:
        intents = json.load(f)

    patterns = []
    tags = []
    max_len = 0
    for intent in intents['intents']:
        
        for pattern in intent['patterns']:
            if(len(pattern) < max_len):
                max_len = len(pattern)
            tag = intent['tag']
            tags.append(tag)
            patterns.append(pattern)

    
    full_df = pd.DataFrame({"text":patterns,"label":tags})
    #full_df['label'] = full_df['label'].apply(lambda x : x -1)
    train, test = train_test_split(full_df, test_size=0.4)
    return full_df,train, test , max_len

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
import pandas as pd
import json
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
def load_data_df(train_file):
    with open(train_file,'r') as f:
        intents = json.load(f)

    patterns = []
    tags = []
    max_len = 0
    for intent in intents['intents']:
        
        for pattern in intent['patterns']:
            if(len(pattern) > max_len):
                max_len = len(pattern)
            tag = intent['tag']
            tags.append(tag)
            patterns.append(pattern)


    full_df = pd.DataFrame({"text":patterns,"label":tags})
    #full_df['label'] = full_df['label'].apply(lambda x : x -1)
    train, test = train_test_split(full_df, test_size=0.4)
    return full_df,train, test , max_len


# utils.py
from torchtext.vocab import FastText

import torch
import torchtext
from torchtext import data
from sklearn.metrics import accuracy_score,f1_score



class Dataset(object):
    def __init__(self, config):
        self.config = config
        self.full_iterator = None
        self.train_iterator = None
        self.test_iterator = None
        self.val_iterator = None
        self.vocab = []
        self.word_embeddings = {}
        self.output_num = None

    def parse_label(self, label):
        '''
        Get the actual labels from label string
        Input:
            label (string) : labels of the form '__label__2'
        Returns:
            label (int) : integer value corresponding to label string
        '''
        return int(label.strip()[-1])

    def get_pandas_df(self, filename):
        '''
        Load the data into Pandas.DataFrame object
        This will be used to convert data to torchtext object
        '''
        with open(filename, 'r') as datafile:
            data = [line.strip().split(',', maxsplit=1) for line in datafile]
            data_text = list(map(lambda x: x[1], data))
            print(len(data_text))
            data_label = list(map(lambda x: self.parse_label(x[0]), data))

        full_df = pd.DataFrame({"text":data_text, "label":data_label})
        return full_df

    def load_data(self, train_file, test_file=None, val_file=None):
        '''
        Loads the data from files
        Sets up iterators for training, validation and test data
        Also create vocabulary and word embeddings based on the data

        Inputs:
            train_file (String): path to training file
            test_file (String): path to test file
            val_file (String): path to validation file
        '''
        # Load data from pd.DataFrame into torchtext.data.Dataset
        full_df , train_df , test_df , max_len = load_data_df(train_file)
    

        NLP = spacy.blank('en')
        
        tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]

        # Creating Field for data
        TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True,fix_length=self.config.max_sen_len, include_lengths=False)
        LABEL = data.Field(sequential=False, use_vocab=False)
        datafields = [("text",TEXT),("label",LABEL)]

   
        self.output_num = len(full_df.label.unique())

        full_examples = [data.Example.fromlist(i, datafields) for i in full_df.values.tolist()]
        full_data = data.Dataset(full_examples, datafields)
        
        train_examples = [data.Example.fromlist(i, datafields) for i in train_df.values.tolist()]
        train_data = data.Dataset(train_examples, datafields)


        test_examples = [data.Example.fromlist(i, datafields) for i in test_df.values.tolist()]
        test_data = data.Dataset(test_examples, datafields)

        # If validation file exists, load it. Otherwise get validation data from training data
        if val_file:
            val_df = self.get_pandas_df(val_file)
            val_examples = [data.Example.fromlist(i, datafields) for i in val_df.values.tolist()]
            val_data = data.Dataset(val_examples, datafields)
        else:
            train_data, val_data = train_data.split(split_ratio=0.8)

        TEXT.build_vocab(train_data, vectors=FastText('simple'))
        #TEXT.build_vocab(full_data, vectors=FastText('simple'))
        
        #TEXT.build_vocab(train_data,vectors=torchtext.vocab.GloVe(name="6B", dim=300, max_vectors=512),max_size=512, min_freq=2)
        self.vocab = TEXT.vocab

        self.full_iterator = data.BucketIterator(
            (full_data),
            batch_size=self.config.batch_size,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=True)
        
        self.train_iterator = data.BucketIterator(
            (train_data),
            batch_size=self.config.batch_size,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=True)

        self.val_iterator, self.test_iterator = data.BucketIterator.splits(
            (val_data, test_data),
            batch_size=self.config.batch_size,
            sort_key=lambda x: len(x.text),
            repeat=False,
            shuffle=False)
        

        print ("Loaded {} training examples".format(len(train_data)))
        print ("Loaded {} test examples".format(len(test_data)))
        print ("Loaded {} validation examples".format(len(val_data)))

def evaluate_model(model, iterator):
    all_preds = []
    all_y = []
    for idx,batch in enumerate(iterator):
        if torch.cuda.is_available():
            x = batch.text.cuda()
        else:
            x = batch.text
        y_pred = model(x)
        predicted = torch.max(y_pred.cpu().data, 1)[1] + 1
        all_preds.extend(predicted.numpy())
        all_y.extend(batch.label.numpy())
    score = accuracy_score(all_y, np.array(all_preds).flatten())
    f1 = f1_score(all_y,np.array(all_preds).flatten(), average='macro')
    
    return score , f1

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
import torch.nn as nn

from   quaternion_layers       import QuaternionLinear
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = QuaternionLinear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = QuaternionLinear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x
def QNorm(x,eps):
    r,i,j,k = torch.chunk(x, chunks=4, dim=-1)
    qnorm = torch.sqrt(r*r + i*i + j*j + k*k + eps)
    r = r/qnorm
    i = i/qnorm
    j = j/qnorm
    k = k/qnorm
        
    return [r,i,j,k]
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model//4
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        
        [r,i,j,k] = QNorm(x,self.eps)
        
        norm_r = self.alpha *r + self.bias
        norm_i = self.alpha *i + self.bias
        norm_j = self.alpha *j + self.bias
        norm_k = self.alpha *k + self.bias
        norm = torch.cat([norm_r,norm_i,norm_j,norm_k],dim=-1)

        return norm
def Qattention(q,k, v, d_k, mask=None, dropout=None):
    [scores_r,scores_i,scores_j,scores_k] = [x/math.sqrt(d_k) for x in quarternion_multiplication(q,k)]
    
    if mask is not None:
        #print("mask",mask)
        mask = mask.unsqueeze(1)
        
        #print(scores_r.shape)
        scores_r = scores_r.masked_fill(mask == 0, -1e9)
        scores_r = F.softmax(scores_r, dim=-1)
        scores_i = scores_i.masked_fill(mask == 0, -1e9)
        scores_i = F.softmax(scores_i, dim=-1)
        scores_j = scores_j.masked_fill(mask == 0, -1e9)
        scores_j = F.softmax(scores_j, dim=-1)
        scores_k = scores_k.masked_fill(mask == 0, -1e9)
        scores_k = F.softmax(scores_k, dim=-1)
    
    if dropout is not None:
        scores_r = dropout(scores_r)
        scores_i = dropout(scores_i)
        scores_j = dropout(scores_j)
        scores_k = dropout(scores_k)

    scores_r = F.softmax(scores_r, dim=-1)
    scores_i = F.softmax(scores_i, dim=-1)
    scores_j = F.softmax(scores_j, dim=-1)
    scores_k = F.softmax(scores_k, dim=-1)
    
 
    v_vals = torch.chunk(v, chunks=4, dim=-1)
    

    scores = torch.cat([scores_r,scores_i,scores_j,scores_k],dim=-1)
    #print(scores.shape)
    #print(v.shape)
    output = quarternion_multiplication(scores,v,transpose=False)
    output = torch.cat(output, dim=-1)
    return output
   
    
class QMultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = QuaternionLinear(d_model, d_model)
        self.v_linear = QuaternionLinear(d_model, d_model)
        self.k_linear = QuaternionLinear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = QuaternionLinear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        
        #print(k.shape,q.shape,v.shape)
        scores = Qattention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output
def quarternion_multiplication(a, b, transpose=True):
    """ Performs hamilton product between two quarternion sequences.
    a = (r,x,y,z)
    b = (r',x',y',z')
    following:
    (rr' - xx' - yy' - zz')  +
    (rx' + xr' + yz' - zy')i +
    (ry' - xz' + yr' + zx')j +
    (rz' + xy' - yx' + zr')k
    """


    ar, ax, ay, az = torch.chunk(a, chunks=4, dim=-1)
    br, bx, by, bz = torch.chunk(b, chunks=4, dim=-1)


    if transpose==True:
        if len(br.shape)>2:
            #r = torch.matmul(br.transpose(-2,-1),ar) - torch.matmul(bx.transpose(-2,-1),ax) - torch.matmul(by.transpose(-2,-1),ay) - torch.matmul(bz.transpose(-2,-1),az)
            #i = torch.matmul(bx.transpose(-2,-1),ar) + torch.matmul(br.transpose(-2,-1),ax) + torch.matmul(bz.transpose(-2,-1),ay) - torch.matmul(by.transpose(-2,-1),az)
            #j = torch.matmul(by.transpose(-2,-1),ar) - torch.matmul(bz.transpose(-2,-1),ax) + torch.matmul(br.transpose(-2,-1),ay) + torch.matmul(bx.transpose(-2,-1),az)
            #k = torch.matmul(bz.transpose(-2,-1),ar) + torch.matmul(by.transpose(-2,-1),ax) - torch.matmul(bx.transpose(-2,-1),ay) + torch.matmul(br.transpose(-2,-1),az)
        
            r = torch.matmul(ar,br.transpose(-2,-1)) - torch.matmul(ax,bx.transpose(-2,-1)) - torch.matmul(ay,by.transpose(-2,-1)) - torch.matmul(az,bz.transpose(-2,-1))
            i = torch.matmul(ar,bx.transpose(-2,-1)) + torch.matmul(ax,br.transpose(-2,-1)) + torch.matmul(ay,bz.transpose(-2,-1)) - torch.matmul(az,by.transpose(-2,-1))
            j = torch.matmul(ar,by.transpose(-2,-1)) - torch.matmul(ax,bz.transpose(-2,-1)) + torch.matmul(ay,br.transpose(-2,-1)) + torch.matmul(az,bx.transpose(-2,-1))
            k = torch.matmul(ar,bz.transpose(-2,-1)) + torch.matmul(ax,by.transpose(-2,-1)) - torch.matmul(ay,bx.transpose(-2,-1)) + torch.matmul(az,br.transpose(-2,-1))
        
            
        else:
            r = torch.matmul(ar, br.t()) - torch.matmul(ax, bx.t()) - torch.matmul(ay, by.t()) - torch.matmul(az, bz.t())
            i = torch.matmul(ar, bx.t()) + torch.matmul(ax, br.t()) + torch.matmul(ay, bz.t()) - torch.matmul(az, by.t())
            j = torch.matmul(ar, by.t()) - torch.matmul(ax, bz.t()) + torch.matmul(ay, br.t()) + torch.matmul(az, bx.t())
            k = torch.matmul(ar, bz.t()) + torch.matmul(ax, by.t()) - torch.matmul(ay, bx.t()) + torch.matmul(az, br.t())
    else:
        r = torch.matmul(ar,br) - torch.matmul(ax,bx) - torch.matmul(ay,by) - torch.matmul(az,bz)
        i = torch.matmul(ar,bx) + torch.matmul(ax,br) + torch.matmul(ay,bz) - torch.matmul(az,by)
        j = torch.matmul(ar,by) - torch.matmul(ax,bz) + torch.matmul(ay,br) + torch.matmul(az,bx)
        k = torch.matmul(ar,bz) + torch.matmul(ax,by) - torch.matmul(ay,bx) + torch.matmul(az,br)
        
    return [r, i, j, k]


In [28]:
# Model.py

import torch
import torch.nn as nn
from copy import deepcopy


class Transformer(nn.Module):
    def __init__(self, config, src_vocab):
        super(Transformer, self).__init__()
        self.config = config

        h, N, dropout = self.config.h, self.config.N, self.config.dropout
        d_model, d_ff = self.config.d_model, self.config.d_ff

        attn = MultiHeadAttention(h, d_model)
        ff = PositionwiseFeedForward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        self.src_vocab = src_vocab
        self.norm = Norm(d_model)
        
        
        
               
        self.src_embed = nn.Sequential(Embeddings(len(self.src_vocab),config.d_model), deepcopy(position)) #Embeddings followed by PE

        # Fully-Connected Layer
        self.fc = nn.Linear(
            self.config.d_model,
            self.config.output_size
        )

        # Softmax non-linearity
        self.softmax = nn.Softmax()

        self.encoder = Encoder(EncoderLayer(config.d_model, deepcopy(attn), deepcopy(ff), dropout), N)
    def forward(self, x):

        embedded_sents = self.src_embed(x.permute(1,0)) # shape = (batch_size, sen_len, d_model)
        
        encoded_sents = self.encoder(embedded_sents)

        # Convert input to (batch_size, d_model) for linear layer

        
        
        final_feature_map = encoded_sents[:,-1,:]


        final_out = self.fc(final_feature_map)

        final_out = self.softmax(final_out)
        
        return final_out

    def add_optimizer(self, optimizer):
        self.optimizer = optimizer

    def add_loss_op(self, loss_op):
        self.loss_op = loss_op

    def reduce_lr(self):
        print("Reducing LR")
        for g in self.optimizer.param_groups:
            g['lr'] = g['lr'] / 2
    def predict(self,text):
        y_pred = self.__call__(x)
        
        return y_pred
    
    def run_epoch(self, train_iterator, val_iterator, epoch):
        train_losses = []
        val_accuracies = []
        losses = []

        # Reduce learning rate as number of epochs increase


        for i, batch in enumerate(train_iterator):


            if torch.cuda.is_available():
                x = batch.text.cuda()
                y = (batch.label-1).type(torch.LongTensor)
                
            else:
                x = batch.text
                y = (batch.label-1).type(torch.LongTensor)
            
            y_pred = self.__call__(x)

            loss = self.loss_op(y_pred, y)
            self.optimizer.zero_grad()
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            self.optimizer.step()

            if i % 100 == 0:
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                print("\tAverage training loss: {:.5f}".format(avg_train_loss))
                losses = []

                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()

        return train_losses, val_accuracies
        
    def run_epoch2(self, train_iterator,val_iterator,epoch):
        losses = []
        for j in range(epoch):
            for i, batch in enumerate(train_iterator):
                if torch.cuda.is_available():
                    x = batch.text.cuda()
                    y = (batch.label-1).type(torch.LongTensor)                
                else:
                    x = batch.text
                    y = (batch.label-1).type(torch.LongTensor)

                y_pred = self.__call__(x)
                print([torch.argmax(y_pred, dim=1),y])
                loss = self.loss_op(y_pred, y)
                self.optimizer.zero_grad()
                
                loss.backward()
                
                self.optimizer.step()
            losses.append(loss.detach().numpy())  
            
            if (j+1) % 1 == 0:
                print (f'Epoch [{j+1}/{epoch}], Loss: {loss.item():.10f}')
                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()
                

                
        return losses
        

In [25]:
# Model.py

import torch
import torch.nn as nn
from copy import deepcopy


class QTransformer(nn.Module):
    def __init__(self, config, src_vocab):
        super(QTransformer, self).__init__()
        self.config = config

        h, N, dropout = self.config.h, self.config.N, self.config.dropout
        d_model, d_ff = self.config.d_model, self.config.d_ff

        attn = QMultiHeadAttention(h, d_model)
        ff = FeedForward(d_model, d_ff, dropout)
        position = PositionalEncoding(d_model, dropout)
        self.src_vocab = src_vocab
        self.norm = Norm(d_model)
        
        
        
               
        self.src_embed = nn.Sequential(Embeddings(len(self.src_vocab),config.d_model), deepcopy(position)) #Embeddings followed by PE

        # Fully-Connected Layer
        self.fc = nn.Linear(
            self.config.d_model,
            self.config.output_size
        )

        # Softmax non-linearity
        self.softmax = nn.Softmax()

        self.encoder = Encoder(EncoderLayer(config.d_model, deepcopy(attn), deepcopy(ff), dropout), N)
    def forward(self, x):

        embedded_sents = self.src_embed(x.permute(1,0)) # shape = (batch_size, sen_len, d_model)
        
        encoded_sents = self.encoder(embedded_sents)

        # Convert input to (batch_size, d_model) for linear layer

        
        
        final_feature_map = encoded_sents[:,-1,:]


        final_out = self.fc(final_feature_map)

        final_out = self.softmax(final_out)
        
        return final_out

    def add_optimizer(self, optimizer):
        self.optimizer = optimizer

    def add_loss_op(self, loss_op):
        self.loss_op = loss_op

    def reduce_lr(self):
        print("Reducing LR")
        for g in self.optimizer.param_groups:
            g['lr'] = g['lr'] / 2
    def predict(self,text):
        y_pred = self.__call__(x)
        
        return y_pred
    
    def run_epoch(self, train_iterator, val_iterator, epoch):
        train_losses = []
        val_accuracies = []
        losses = []

        # Reduce learning rate as number of epochs increase


        for i, batch in enumerate(train_iterator):


            if torch.cuda.is_available():
                x = batch.text.cuda()
                y = (batch.label-1).type(torch.LongTensor)
                
            else:
                x = batch.text
                y = (batch.label-1).type(torch.LongTensor)
            
            y_pred = self.__call__(x)

            loss = self.loss_op(y_pred, y)
            self.optimizer.zero_grad()
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            self.optimizer.step()

            if i % 100 == 0:
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                print("\tAverage training loss: {:.5f}".format(avg_train_loss))
                losses = []

                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()

        return train_losses, val_accuracies
        
    def run_epoch2(self, train_iterator,val_iterator,epoch):
        losses = []
        for j in range(epoch):
            for i, batch in enumerate(train_iterator):
                if torch.cuda.is_available():
                    x = batch.text.cuda()
                    y = (batch.label-1).type(torch.LongTensor)                
                else:
                    x = batch.text
                    y = (batch.label-1).type(torch.LongTensor)

                y_pred = self.__call__(x)
                print([torch.argmax(y_pred, dim=1),y])
                loss = self.loss_op(y_pred, y)
                self.optimizer.zero_grad()
                
                loss.backward()
                
                self.optimizer.step()
            losses.append(loss.detach().numpy())  
            
            if (j+1) % 1 == 0:
                print (f'Epoch [{j+1}/{epoch}], Loss: {loss.item():.10f}')
                # Evalute Accuracy on validation set
                val_accuracy,f1 = evaluate_model(self, val_iterator)
                print("\tVal Accuracy: {:.4f}".format(val_accuracy))
                print("\tf1: {:.4f}".format(f1))
                
                self.train()
                

                
        return losses
        

In [33]:
config = Config()
config.batch_size =1
config.d_ff = 1024
config.lr = 0.0001
config.max_epochs = 300
dataset = Dataset(config)

train_file = '/Users/mac/Desktop/test/test.json'
dataset.load_data(train_file)
dataset.config.output_size = dataset.output_num

net_r = Transformer(config, dataset.vocab)
net_q = QTransformer(config, dataset.vocab)

Loaded 41 training examples
Loaded 34 test examples
Loaded 10 validation examples


In [35]:
nb_param_q = sum(p.numel() for p in net_q.parameters() if p.requires_grad)
nb_param_r = sum(p.numel() for p in net_r.parameters() if p.requires_grad)


print("Models Infos --------------------")
print("(QTransformer) Number of trainable parameters : "+str(nb_param_q))
print("(Transformer)  Number of trainable parameters : "+str(nb_param_r))

Models Infos --------------------
(QTransformer) Number of trainable parameters : 3304728
(Transformer)  Number of trainable parameters : 19045656


In [ ]:
# import sys  
import torch.optim as optim
from torch import nn
import torch
import spacy
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
config = Config()
config.batch_size =1
config.d_ff = 1024
config.lr = 0.0001
config.max_epochs = 500
dataset = Dataset(config)

train_file = '/Users/mac/Desktop/test/test.json'
dataset.load_data(train_file)
dataset.config.output_size = dataset.output_num


print(dataset.vocab.vectors.size())
# Create Model with specified optimizer and loss function
##############################################################


model = QTransformer(config, dataset.vocab)
optimizer = optim.Adam(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)

model.run_epoch2(dataset.full_iterator, dataset.val_iterator,config.max_epochs)

Loaded 41 training examples
Loaded 34 test examples
Loaded 10 validation examples
torch.Size([134, 300])
[tensor([9]), tensor([6])]
[tensor([8]), tensor([12])]
[tensor([9]), tensor([21])]
[tensor([9]), tensor([15])]
[tensor([9]), tensor([5])]
[tensor([9]), tensor([20])]
[tensor([9]), tensor([12])]
[tensor([9]), tensor([16])]
[tensor([5]), tensor([18])]
[tensor([9]), tensor([16])]
[tensor([9]), tensor([22])]
[tensor([9]), tensor([1])]
[tensor([5]), tensor([16])]
[tensor([9]), tensor([6])]
[tensor([9]), tensor([2])]
[tensor([5]), tensor([11])]
[tensor([5]), tensor([4])]
[tensor([9]), tensor([0])]
[tensor([5]), tensor([19])]
[tensor([5]), tensor([5])]
[tensor([5]), tensor([23])]
[tensor([5]), tensor([13])]
[tensor([5]), tensor([15])]
[tensor([5]), tensor([2])]
[tensor([5]), tensor([19])]
[tensor([5]), tensor([23])]
[tensor([5]), tensor([17])]
[tensor([5]), tensor([6])]
[tensor([5]), tensor([18])]
[tensor([5]), tensor([16])]
[tensor([5]), tensor([4])]
[tensor([5]), tensor([2])]
[tensor([5]

In [ ]:
# import sys  
import torch.optim as optim
from torch import nn
import torch
import spacy
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
if __name__=='__main__':
    config = Config()
    config.batch_size =1
    config.d_ff = 1024
    config.lr = 0.0001
    config.max_epochs = 50
    dataset = Dataset(config)

    train_file = '/Users/mac/Desktop/test/test.json'
    dataset.load_data(train_file)
    dataset.config.output_size = dataset.output_num

    
    print(dataset.vocab.vectors.size())
    # Create Model with specified optimizer and loss function
    ##############################################################

    model = Transformer(config, dataset.vocab)
    
    #for p in model.parameters():
     #   if p.dim() > 1: nn.init.xavier_uniform(p)

    if torch.cuda.is_available():
        model.cuda()
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
    NLLLoss = nn.NLLLoss()
    model.add_optimizer(optimizer)
    model.add_loss_op(NLLLoss)
    ##############################################################

    train_losses = []
    val_accuracies = []

    for i in range(config.max_epochs):
        print ("Epoch: {}".format(i))
        train_loss,val_accuracy = model.run_epoch(dataset.full_iterator, dataset.val_iterator, i)
        train_losses.append(train_loss)
        val_accuracies.append(val_accuracy)

    train_acc,train_f1 = evaluate_model(model, dataset.train_iterator)
    val_acc,val_f1 = evaluate_model(model, dataset.val_iterator)
    test_acc,test_f1 = evaluate_model(model, dataset.test_iterator)

    print ('Final Training Accuracy: {:.4f}'.format(train_acc))
    print ('Final Training f1: {:.4f}'.format(train_f1))
    
    print ('Final Validation Accuracy: {:.4f}'.format(val_acc))
    print ('Final Training f1: {:.4f}'.format(val_f1))

    print ('Final Test Accuracy: {:.4f}'.format(test_acc))
    print ('Final Training f1: {:.4f}'.format(test_f1))


In [ ]:
#SAVE MODEL

torch.save(model, '/Users/mac/Desktop/test/model.pth')
model= torch.load('/Users/mac/Desktop/test/model.pth')

model.eval()



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bot_name = "QP"
print("Hi! I am Fernando")
print("I am here to help you feel free to chat with me:-")
print("Let's chat! (type 'quit' to exit)")
print("Please write your question with keyword below")

with open('/Users/mac/Desktop/SCIENTIFIC RESEARCH/main QA.json', 'r') as json_data:
    main_intents = json.load(json_data)
corpse = []
responses = []
for intent in main_intents['intents']:
    tag = intent['tag']
    response = intent['responses']
    print(tag+"\n")
    corpse.append(tag)# here we are appending the word with its tag
    responses.append(response)

In [ ]:
from nltk.tokenize import regexp_tokenize
def tokenize(sentence):
    return regexp_tokenize(sentence, pattern="\w+")

while True:
    sentence = input("You: ")
    if(any(sentence.lower()==item.lower() for item in ["quit","finish","over","bye","goodbye"])):
        print(f"{bot_name}: Goodbye , have a nice day")
        break

    similarity = []
    for i in corpse:
        similarity.append(score_words(sentence,i))
    #print(similarity)
    
    if(max(similarity) > 0.5 and len(tokenize(sentence))==1 ):
        print(f"{bot_name}: "+responses[similarity.index(max(similarity))][0])
    else:
    ## tensor part
        NLP = spacy.load('en_core_web_sm')
        tokenizer = lambda sent: [x.text for x in NLP.tokenizer(sent) if x.text != " "]
        token = tokenizer(sentence)
        sequenced=[]
        for i in range(len(token)):
            sequenced.append(model.src_vocab[token[i]])

        while(len(sequenced)!=model.config.max_sen_len):
            sequenced.append(0)

        array = np.array(sequenced)
        transposed_array = array.T
        sequenced = transposed_array.tolist()
        tensor = []
        tensor.append(sequenced)
        output = model(torch.tensor(tensor)).max(1).indices.tolist()

        occurence = Counter(output).most_common()
        sum = 0 
        for i in range(len(occurence)):
            sum += occurence[i][1]
        prob = occurence[0][1]/sum

        tag = occurence[0][0]
        ##
        with open('/Users/mac/Desktop/test/test.json', 'r') as json_data:
            intents = json.load(json_data)
        if prob > 0.7:
                for intent in intents['intents']:
                    if tag == intent["tag"]:
                        print(f"{bot_name}: {random.choice(intent['responses'])}")
        else:
            print(f"{bot_name}: Sorry I am unable to Process Your Request")
            print(f"{bot_name}: You may find the way forward in https://en.itmo.ru/en/viewjep/2/5/Big_Data_and_Machine_Learning.htm")

In [ ]:
token = lambda word: nlp(word)[0]

token('i am fer')
